# **Notebook 2 - Compute DiverCity**

---

## **Overview**
This notebook computes the **DiverCity measure** for a selected city using the preprocessed road network generated in *Notebook 1*.  
It implements the **Path Penalization (PP)** algorithm to generate near-shortest alternative routes and computes the **DiverCity measure**, quantifying the potential route diversification between origin–destination pairs.

Following the DiverCity framework, the computation involves:
- Loading the city’s processed road network.  
- Defining penalization (`p`) and tolerance threshold (`ε`) parameters for near-shortest route generation.  
- Computing alternative routes using **Path Penalization**.  
- Evaluating route diversification.

---

## **DiverCity Definition**

For each trip between an origin `u` and a destination `v`, DiverCity quantifies how many **distinct near-shortest routes** exist and **how spatially diverse** they are.

It is defined as:

$D(u,v) = S(\text{NSR}(u,v)) \cdot |\text{NSR}(u,v)|$

where:

* **NSR(u,v)** is the set of *near-shortest routes* between `u` and `v`, i.e., routes whose travel cost does not exceed that of the fastest route by more than *ε*%.
* **S(NSR(u,v)) = 1 − J(NSR(u,v))**, where *J* is the average *weighted Jaccard similarity* among all route pairs, capturing their spatial overlap.
* A high `S` indicates spatially diverse alternatives, while a high `|NSR|` indicates numerous near-shortest routes.

Hence, **DiverCity** increases when a trip has *many distinct* and *loosely overlapping* feasible routes.

---

## **Configuration and Parameters**
The following parameters control the DiverCity computation:

| Parameter | Description |
|------------|-------------|
| `city_name` | Name of the city under analysis. |
| `city_center` | Geographic coordinates (latitude, longitude) of the city center. |
| `exp_id` | Unique identifier for the experiment (used for file naming). |
| `list_p` | List of penalization factors for the Path Penalization algorithm. |
| `list_eps` | List of ε values defining the deviation threshold for Near-Shortest Routes (NSR). |
| `k` | Number of alternative routes to generate per OD pair. |
| `attribute` | Edge attribute used for optimization (e.g., `"travel_time"` or `"length"`). |
| `radius_range` | Range of radii (in km) used for radial sampling of OD pairs. |
| `samples_per_circle` | Number of sampled origins and destinations per radial distance. |
| `save_routes` | Boolean flag indicating whether to store generated routes for post-analysis. |

---

### **Parameters Used in This Study**

The experiments in this notebook follow the configuration adopted in the DiverCity study to ensure consistency and reproducibility across all cities.  
The parameters below control the generation of near-shortest routes and the computation of route diversification metrics.

| Parameter | Symbol | Description | Values Used |
|------------|---------|--------------|--------------|
| **Number of alternative routes** | *k* | Maximum number of near-shortest routes computed for each origin–destination pair. | 15 |
| **Extraction radius** | — | Maximum distance from the city center for radial sampling (in km). | 30 km |
| **Penalization factor** | *p* | Edge weight penalization applied in the Path Penalization (PP) algorithm to encourage exploration of alternative paths. | 0.05, 0.10, 0.20, 0.30 |
| **Tolerance threshold** | *ε* | Maximum allowed deviation (in % of travel cost) from the fastest route to define a Near-Shortest Route (NSR). | 0.1 (10%), 0.2 (20%), 0.3 (30%), 0.4 (40%), 0.5 (50%) |

The values used for the main results are **p = 0.1**, **ε = 0.3**, and **k = 10**.

---

All results are stored in the designated **output folder** for reuse in subsequent notebooks (*Attractor Analysis* and *Visualization*).


In [ ]:
from divercity_utils import compute_divercity_city
from trip_measures import compute_traveltime
import pandas as pd
from my_utils import create_folder_if_not_exists

## **1. Setting the Experiments**

This section defines the configuration for the DiverCity computation experiment.  
Each experiment is identified by a unique ID and associated with a specific city.  
Parameters control the generation of origin–destination pairs, the Path Penalization (PP) process, and the thresholds used to identify Near-Shortest Routes (NSR).

All results will be saved in the corresponding output directory for later analysis and visualization.

---

### **1.1 Parameters**
The following parameters can be modified to reproduce different experimental settings:
- **Experiment ID** - Unique tag used for naming result files.  
- **City Selection** - Either specify coordinates manually or load them from `city_info.csv`.  
- **Sampling Settings** - Define how origin–destination pairs are distributed radially from the city center.  
- **DiverCity Parameters** - Control penalization (`p`), tolerance (`ε`), number of routes (`k`), and travel attribute.  


In [ ]:
# ------------------------------------------------------------
# Experiment Configuration
# ------------------------------------------------------------

# Unique identifier for the experiment (used in file naming and result tracking)
exp_id = "exp_osm"

# ------------------------------------------------------------
# City Selection
# ------------------------------------------------------------

# City identifier (used for loading the network and organizing output files)
city_name = "milan"

# Option 1 — Recommended: load coordinates from a CSV file
# The file '../data/city_info.csv' must contain at least the columns: ["city", "lat", "lng"]
df_cities = pd.read_csv("../data/city_info.csv")
city_center = df_cities[df_cities["city"] == city_name][["lat", "lng"]].values[0]

# Option 2 — Manual specification (uncomment and edit if needed)
# city_center = (45.4642, 9.19)  # Example for Milan

# ------------------------------------------------------------
# Sampling Parameters
# ------------------------------------------------------------

min_radius_km = 1                     # Minimum radial distance from city center (km)
max_radius_km = 10                    # Maximum radial distance (should not exceed the downloaded network radius)
radius_step_km = 1                    # Distance between concentric sampling rings (km)
n_samples_circle = 8                  # Number of sampled nodes (origins/destinations) per circle

# ------------------------------------------------------------
# DiverCity Parameters
# ------------------------------------------------------------

list_p = [0.1, 0.2, 0.3]              # Path penalization factors for the Path Penalization (PP) algorithm
list_eps = [0.1, 0.3, 0.5]            # Epsilon thresholds for Near-Shortest Routes (fractional deviation from fastest path)
k = 5                                 # Number of alternative routes to compute for each OD pair
attribute = "travel_time"             # Edge attribute used for optimization (e.g., 'travel_time' or 'length')

# ------------------------------------------------------------
# Output Settings
# ------------------------------------------------------------

save_routes = 1                       # Whether to store generated routes (1=True, 0=False)
directory_path = "../data/results/"   # Directory where all results will be saved

# Ensure the output directory exists
create_folder_if_not_exists(directory_path)

print(f"Experiment '{exp_id}' configured successfully for {city_name.title()}")

### **1.2 Launch the Standard Execution (Original Speed)**

This step runs the DiverCity computation for the selected city under standard speed conditions.

⚠️ Note: This step may take some time to complete, depending on the network size, the number of origin–destination pairs, and the number of parallel jobs (`njobs`).

In [ ]:
reducespeed = 1

compute_divercity_city(city_name, city_center, exp_id, list_p, list_eps, k, attribute, 
                                reducespeed, min_radius_km, max_radius_km, radius_step_km, 
                                n_samples_circle, save_routes, njobs=10)

### **1.3 Launch the Execution with Speed Reductions on Mobility Attractors**

In this experiment, we simulate the effect of **speed limit reductions** on major mobility attractors (e.g., highways, ring roads).  
For each reduction factor in `list_speed_reductions`, the function `compute_divercity_city()` re-runs the DiverCity computation, updating the speed limits of attractor edges proportionally.

For example, a value of **0.8** indicates that the speed limits on mobility attractors is set to **80% of their original value**.

⚠️ Note: This step may take some time to complete, depending on the network size, the number of origin–destination pairs, and the number of parallel jobs (`njobs`).

In [ ]:
list_speed_reductions = [0.9, 0.5, 0.4, 0.3, 0.2, 0.1]  # Reduction factors from mild (0.9x) to strong (0.1x)

for reducespeed in list_speed_reductions:
    print(f"\nRunning DiverCity computation with speed reduction factor = {reducespeed}")
    compute_divercity_city(city_name, city_center, exp_id, list_p, list_eps, k, attribute, 
                                    reducespeed, min_radius_km, max_radius_km, radius_step_km, 
                                    n_samples_circle, save_routes, njobs=10)

## **2. Computing Routes-Related Information (Travel Times)**

After computing DiverCity under different speed reduction scenarios, this step evaluates **travel-time–related information** for each configuration.  
The function `compute_traveltime()` aggregates and compares travel times across all speed settings and penalization factors.

These results are used to reproduce the figures and analyses presented in the paper (e.g., *Figure 3b–e*), where travel times are compared against DiverCity improvements under various speed reduction levels.

In [ ]:
for p in list_p:
    
    print(f"\nComputing travel-time information for penalization factor p = {p}")
    
    compute_traveltime(
        city_name,
        [int(x * 100) for x in list_speed_reductions],  # Convert reduction factors (e.g., 0.5 → 50)
        p, exp_id)